In [1]:
import surrogate_model
import argparse
import numpy as np
import pandas as pd
import src.dynamic_pipeline as dynamic_pipeline

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./data/concrete.csv')
# df = dynamic_pipeline.preprocess_dynamic(df)

In [3]:
ret_dict = dynamic_pipeline.preprocess_dynamic(df)

제거된 컬럼이 없습니다.
cement: 비대칭 분포로 확인됨. IQR 방식 적용.
slag: 비대칭 분포로 확인됨. IQR 방식 적용.
ash: 비대칭 분포로 확인됨. IQR 방식 적용.
water: 비대칭 분포로 확인됨. IQR 방식 적용.
superplastic: 비대칭 분포로 확인됨. IQR 방식 적용.
coarseagg: 비대칭 분포로 확인됨. IQR 방식 적용.
fineagg: 비대칭 분포로 확인됨. IQR 방식 적용.
age: 비대칭 분포로 확인됨. IQR 방식 적용.
strength: 비대칭 분포로 확인됨. IQR 방식 적용.
cement: 값 범위 중요 → MinMaxScaler 적용
slag: 값 범위 중요 → MinMaxScaler 적용
ash: 값 범위 중요 → MinMaxScaler 적용
water: 값 범위 중요 → MinMaxScaler 적용
superplastic: 값 범위 중요 → MinMaxScaler 적용
coarseagg: 값 범위 중요 → MinMaxScaler 적용
fineagg: 값 범위 중요 → MinMaxScaler 적용
age: 비대칭 분포 → RobustScaler 적용
strength: 값 범위 중요 → MinMaxScaler 적용


In [4]:
ret_dict['processed_df'].to_csv('./data/temp.csv', index=False)

In [5]:
# args = argparse.Namespace(
#     target = ['strength'],
#     data_path='./data/temp.csv',
#     model='catboost',
#     prj_id = 40,
#     seed=40
# )
# surrogate_model.main(args, ret_dict['scalers'])

In [6]:
args = argparse.Namespace(
    target = ['strength'],
    data_path='./data/temp.csv',
    model='tabpfn',
    prj_id = 40,
    seed=40
)
surrogate_model.main(args, ret_dict['scalers'])

2025-02-03 11:57:37,754 - INFO - X_train.shape: (740, 8)
2025-02-03 11:57:37,755 - INFO - X_test.shape: (186, 8)
2025-02-03 11:57:37,755 - INFO - y_train.shape: (740, 1)
2025-02-03 11:57:37,756 - INFO - y_test.shape: (186, 1)
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(186, 1)
(186, 1)
      rmse       mae       r2
0  0.05628  0.033254  0.92657
(186, 1) (186, 1)
(186, 1)


In [7]:
import os

In [8]:
os.listdir('./prj/40/surrogate_model')

['importance.csv', 'eval.csv', 'surrogate_model.csv', 'model.cbm', 'model.pkl']

In [9]:
import search_model

In [10]:
controll_range = df.drop(columns=['strength']).columns.tolist()
controll_range = {col: (df[col].min(), df[col].max()) for col in controll_range}


In [11]:
controll_range

{'cement': (102.0, 540.0),
 'slag': (0.0, 359.4),
 'ash': (0.0, 200.1),
 'water': (121.8, 247.0),
 'superplastic': (0.0, 32.2),
 'coarseagg': (801.0, 1145.0),
 'fineagg': (594.0, 992.6),
 'age': (1, 365)}

In [12]:
print(df['strength'].min(),'~',df['strength'].max())

2.33 ~ 82.6


In [13]:
from src.datasets.data_loader import load_data

In [14]:
df_scaled = load_data(args.data_path)
X = df_scaled.drop(columns=args.target)
y = df_scaled[args.target]
x_col_list = X.columns.tolist()

X_train,y_train = X.to_numpy(),y.to_numpy()


In [15]:
scalers = ret_dict['scalers']

In [16]:
# user_req = scalers['strength'].transform(np.array(args.user_request_target[0]).reshape(-1,1))

In [17]:
ur = []
ur.append(np.array(40.0).reshape(-1,1)[0])
ur = np.array(ur)

In [18]:
ur

array([[40.]])

In [19]:
x,y = search_model.find_top_k_similar_with_user_request(ur,X_train,y_train,k=50)

In [20]:
x.shape
y.shape


(50, 1)

In [21]:
from src.datasets.data_loader import load_data

In [22]:
args = argparse.Namespace(
    model='catboost',
    search_model='k_means',
    data_path='./data/temp.csv',
    controll_name=['cement', 'slag', 'ash', 'water', 'superplastic', 'coarseagg', 'fineagg', 'age'],
    controll_range= controll_range,
    target=['strength'],
    importance={'cement': 1,
                'water': 2,
    },
    optimize={'cement': 'minimize',
                'water': 'minimize',
    },
    prj_id=40,
    seed=40,
    user_request_target=[40.0]
)

In [23]:
# %%time 
x_opt = search_model.main(args,ret_dict['scalers'])

/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/utils/valida

x_min :  [ 0.          0.          0.         -0.05148515  0.          0.
 -0.03235294 -1.28571429]
x_max :  [ 1.          1.05057001  1.          1.18811881  1.45701357  1.
  1.14       16.04761905]


0it [00:00, ?it/s]WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide at least 2574 training points
WARNING clustering 200 points to 66 centroids: please provide 

search model 소요 시간: 48.6532초
(50, 1)


In [24]:
x_opt

,pred_x,test_x,test_y,pred_y
0,"[539.9999999999995, 175.7978009275822, 106.627...","[259.9, 100.6, 78.39999999999999, 170.6, 10.39...",[[49.77]],49.770402
1,"[284.4546752247298, 192.50952390589435, 101.68...","[260.0, 101.0, 78.0, 171.0, 9.999999999999998,...",[[49.77]],49.769998
2,"[121.21721750048685, 126.04398159905115, 11.71...","[266.2, 112.29999999999998, 87.49999999999999,...",[[39.42]],39.421218
3,"[271.6773148576471, 0.0, 59.792506081788616, 2...","[266.0, 112.00000000000001, 87.0, 178.0, 9.999...",[[39.42]],39.420560
4,"[200.3620981076762, 352.874001703317, 194.4692...","[276.0, 116.0, 90.0, 180.0, 9.0, 870.0, 768.0,...",[[44.28]],44.279938
5,"[332.2671074879337, 124.85202109938686, 150.82...","[276.4, 116.0, 90.3, 179.6, 8.899999999999999,...",[[44.28]],44.280942
6,"[240.5629750912339, 44.44696703610959, 68.4607...","[287.29999999999995, 120.49999999999999, 93.9,...",[[43.8]],43.800648
7,"[203.77387693645878, 248.52193002474445, 67.59...","[287.0, 120.99999999999999, 94.0, 188.0, 9.0, ...",[[41.94]],41.938883
8,"[141.70490875797904, 359.4, 96.09661059612317,...","[255.29999999999998, 98.79999999999998, 77.0, ...",[[33.8]],33.800028
9,"[102.0, 157.15119762205617, 12.336415786419783...","[255.0, 98.99999999999999, 77.0, 189.0, 5.9999...",[[33.8]],33.799468
